In [7]:
# 임시 파일 확인

import pandas as pd

df = pd.read_csv('risk_scores2.csv')
df.head(7)

,id,session_id,username,topic,risk_score,created_at
0,1,433d7d51-08a1-44d9-b4ed-28f6fa4aed0e,bbb,분노/짜증,3,2025-10-28 1:55
1,2,433d7d51-08a1-44d9-b4ed-28f6fa4aed0e,bbb,형제자매,0,2025-10-28 1:55
2,3,433d7d51-08a1-44d9-b4ed-28f6fa4aed0e,bbb,걱정,4,2025-10-28 1:55
3,4,433d7d51-08a1-44d9-b4ed-28f6fa4aed0e,bbb,정서학대,0,2025-10-28 1:55
4,5,433d7d51-08a1-44d9-b4ed-28f6fa4aed0e,bbb,신체학대,0,2025-10-28 1:55
5,6,433d7d51-08a1-44d9-b4ed-28f6fa4aed0e,bbb,성학대,0,2025-10-28 1:55
6,7,433d7d51-08a1-44d9-b4ed-28f6fa4aed0e,bbb,가정폭력,3,2025-10-28 1:55


#### 

In [5]:
import pandas as pd
import numpy as np
from pathlib import Path

# ===== 0) 파일 경로 =====
CSV_PATH = Path("risk_scores2.csv")  # 환경에 맞게 수정

# ===== 1) 사용 토픽 정의 =====
ALL_11_TOPICS = [
    "정서학대", "분노/짜증", "트라우마", "걱정",
    "신체학대", "성학대",
    "가정폭력", "형제자매", "가출",
    "학교폭력", "자해/자살",
]

FINAL_6_TOPICS = ["정서", "신체학대", "성학대", "가정", "학교폭력", "자해/자살"]

EMOTION_MEMBERS = ["정서학대", "분노/짜증", "트라우마", "걱정"]
FAMILY_MEMBERS  = ["가정폭력", "형제자매", "가출"]

def round2(x):
    return round(float(x), 2) if pd.notna(x) else np.nan

def safe_mean(row, members):
    vals = [row.get(c) for c in members if c in row and pd.notna(row.get(c))]
    if not vals:
        return np.nan
    return round(float(np.mean(vals)), 2)

def main(csv_path=CSV_PATH):
    # 1) 로드 및 불필요 토픽 제거
    df = pd.read_csv(csv_path)
    df = df[df["topic"].isin(ALL_11_TOPICS)].copy()

    # 2) 세션·토픽 중복시 risk_score 최대값 사용
    agg_session_topic = (
        df.groupby(["username", "session_id", "topic"], as_index=False)["risk_score"].max()
    )

    # 3) 피벗 (세션 단위)
    pivot = agg_session_topic.pivot_table(
        index=["username", "session_id"],
        columns="topic",
        values="risk_score",
        aggfunc="max"
    ).reset_index()
    pivot.columns.name = None

    # 4) 그룹 평균 계산(정서, 가정)
    pivot["정서"] = pivot.apply(lambda r: safe_mean(r, EMOTION_MEMBERS), axis=1)
    pivot["가정"] = pivot.apply(lambda r: safe_mean(r, FAMILY_MEMBERS), axis=1)

    # 5) 단일 주제 반올림
    for t in ["신체학대", "성학대", "학교폭력", "자해/자살"]:
        if t in pivot.columns:
            pivot[t] = pivot[t].apply(round2)

    # 6) 누락 컬럼 채우기 (없으면 NaN)
    for t in FINAL_6_TOPICS:
        if t not in pivot.columns:
            pivot[t] = np.nan

    # 7) 세션 단위 Top-3 (토픽별)
    rows = []
    for t in FINAL_6_TOPICS:
        sub = (
            pivot[["username", "session_id", t]]
            .dropna(subset=[t])
            .sort_values(t, ascending=False)
            .head(3)
            .rename(columns={t: "risk_score"})
            .assign(topic=t)
        )
        rows.append(sub[["topic", "username", "session_id", "risk_score"]])

    top3_table = pd.concat(rows, ignore_index=True)

    # 8) 토픽 내 순위(rank) 부여 -> 이후 정렬
    top3_table = (
        top3_table
        .sort_values(["topic", "risk_score"], ascending=[True, False])
        .assign(rank=lambda d: d.groupby("topic").cumcount() + 1)
        .loc[:, ["rank", "topic", "username", "session_id", "risk_score"]]
        .reset_index(drop=True)
    )

    return top3_table

if __name__ == "__main__":
    result = main(CSV_PATH)
    print(result.to_string(index=False))


 rank topic username                           session_id  risk_score
    1    가정      bbb 433d7d51-08a1-44d9-b4ed-28f6fa4aed0e        4.00
    2    가정      fff                             ffffffff        3.33
    3    가정      aaa c7cf0cfd-a866-42f4-8f0c-592439d69c7f        0.67
    1   성학대      ddd                              ddddddd        6.00
    2   성학대      aaa c7cf0cfd-a866-42f4-8f0c-592439d69c7f        4.00
    3   성학대      bbb 433d7d51-08a1-44d9-b4ed-28f6fa4aed0e        0.00
    1  신체학대      aaa c7cf0cfd-a866-42f4-8f0c-592439d69c7f        2.00
    2  신체학대      bbb 433d7d51-08a1-44d9-b4ed-28f6fa4aed0e        0.00
    3  신체학대      ccc                             cccccccc        0.00
    1 자해/자살      bbb 433d7d51-08a1-44d9-b4ed-28f6fa4aed0e        5.00
    2 자해/자살      ccc                             cccccccc        2.00
    3 자해/자살      ddd                              ddddddd        2.00
    1    정서      bbb 433d7d51-08a1-44d9-b4ed-28f6fa4aed0e        2.50
    2    정서      fff